In [1]:
import pandas as pd
import re
import numpy as np
import functions

In [396]:
cur = functions.connectToPs()

Connecting to Postgree...

<connection object at 0x00000257D4ACD378; dsn: 'user=oxbigdatabi password=xxx dbname=bigdatabi host=bigdatabi.ckq5duwtue6h.eu-central-1.rds.amazonaws.com', closed: 0>


# VENTAS

Las diferencias con respecto a la cartera de Marco vienen por: 
- Sanitarios Trucho que está modificado directamente en qlik como 1-GRATIS
- ***no me ha quedado muy claro que hacer con jaioje@yahoo.es*** habría que meterlo por fuera 

In [5]:
df = functions.ventas(cur)
df.head(2)

,segmento,assetid,product_name,purchase_date,canal_venta,cif,purchase_month,purchase_year,Family,Velocity,isNeba,Total
0,3-NOTRIAL,02i0N000006MOzCQAW,X UCom Licencia de Colaboración,2019-02-21 00:00:01,dho,B47764840,2,2019,X UCom,,0,1
1,3-NOTRIAL,02i0N000006M6bnQAC,X Music4B,2019-02-12 00:00:01,dho,B85204915,2,2019,X Music,,0,1


In [8]:
df.dtypes

segmento                  object
assetid                   object
product_name              object
purchase_date     datetime64[ns]
canal_venta               object
cif                       object
purchase_month             int64
purchase_year              int64
Family                    object
Velocity                  object
isNeba                     int64
Total                      int64
dtype: object

In [4]:
vta_fam_ass = df[['Total']].groupby(by=[df.purchase_year,df.purchase_month,df.Family]).sum().reset_index()

In [5]:
vta_fam_ass.pivot_table('Total','Family',['purchase_year','purchase_month'], fill_value= 0, margins=True, aggfunc= sum)

purchase_year  2018            2019                                          \
purchase_month    9 10  11  12    1   2   3   4   5    6    7   8    9   10   
Family                                                                        
X Fiber           0  0   4   8    7  10   2   8  13   17   17   4   20   17   
X Music           2  3   2   3    4   7   2   0   6    0    0   0    5    0   
X Privacy         0  0   5   4    0  12  14   4  12   13    6   2    9    6   
X Protection      0  0   0   4    2   5  25  13   4   29   20   5   20   28   
X Security        0  0   0   2    2   0   0   0   5   13    5   0   53   28   
X UCom            0  0  13  13   22  22  11  16  22   65   57  12   59   94   
All               2  3  24  34   37  56  54  41  62  137  105  23  166  173   

purchase_year            2020   All  
purchase_month   11   12    1        
Family                               
X Fiber           2    3    4   136  
X Music           0    0    0    34  
X Privacy         0    2   11   100  
X Protection     28    4    0   187  
X Security        2    2    0   112  
X UCom           78  120   27   631  
All             110  131   42  1200

In [6]:
vta_fam_cif = df[['cif']].groupby(by=[df.purchase_year,df.purchase_month,df.Family]).nunique().reset_index()

In [8]:
vta_fam_cif.pivot_table('cif','Family',['purchase_year','purchase_month'], fill_value= 0, margins=True, aggfunc= sum)

purchase_year  2018            2019                                          \
purchase_month    9 10  11  12    1   2   3   4   5   6   7   8   9  10  11   
Family                                                                        
X Fiber           0  0   3   8    7   7   2   8   6   9  16   4   8   1   2   
X Music           1  3   2   3    2   4   2   0   6   0   0   0   3   0   0   
X Privacy         0  0   2   2    0   5   7   2   2   5   3   1   3   2   0   
X Protection      0  0   0   4    2   5   5   8   4  15  17   5  12  17  12   
X Security        0  0   0   2    2   0   0   0   2   5   3   0   4   3   1   
X UCom            0  0   4   4    6   6   7   5  11  19  17   6  22  27  28   
All               1  3  11  23   19  27  23  23  31  53  56  16  52  50  43   

purchase_year      2020  All  
purchase_month  12    1       
Family                        
X Fiber          3    1   85  
X Music          0    0   26  
X Privacy        1    3   38  
X Protection     4    0  110  
X Security       1    0   23  
X UCom          29    9  200  
All             38   13  482

In [17]:
vta_cif =  df[['cif']].groupby(by=[df.purchase_year,df.purchase_month]).nunique().reset_index().T

In [18]:
vta_cif

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
purchase_year,2018,2018,2018,2018,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2020
purchase_month,9,10,11,12,1,2,3,4,5,6,7,8,9,10,11,12,1
cif,1,3,11,19,14,25,22,19,25,45,51,16,51,49,42,38,13


In [108]:
# Assets borrados en Salesforce
# df[(df['assetid']=='02i0N00000Pf77zQAB')|(df['assetid']=='02i0N00000Pf780QAB')]
# comp = df[(df['purchase_month']==12) & (df['purchase_year']==2019)]
# comp.to_csv('./comp_2019_12.csv',index=False)
# df[(df['purchase_month']==9)&(df['purchase_year']==2019)]['cif'].unique()

# ALTAS

Las diferencias con respecto a la cartera de Marco son:
- rfb - NEBA 50 de 2010 fórmulas y recetas en la cartera de Marco tiene fecha 2020-01 y en cartera_xbo 2019-03
- Modificar en la cartera de marco para que entre el X UCom que tiene en la descripción 'portabilidad a cancerlar'
- rbf - X UCom de Estudio de Ingeniería Lynka.SL en la cartera de Marco tiene fecha 2019-12 y en la cartera_xbo 2020-01
- 02i0N00000Pf77zQAB y 02i0N00000Pf780QAB han sido borrados por SF y no está actualizado en la cartera de Marco 


##### VERIFICAR REGLA

- **Hay que descartar aquellos assets cuyo rbs y baja sean en la misma fecha ** 


In [387]:
altas = functions.altas(cur)

In [4]:
altas.dtypes

segmento                object
assetid                 object
product_name            object
rfb_date        datetime64[ns]
canal_venta             object
cif                     object
alta_month               int64
alta_year                int64
Family                  object
Velocity                object
isNeba                   int64
Total                    int64
dtype: object

In [5]:
altas.shape

(1082, 12)

In [113]:
altas.head(1)

,segmento,assetid,product_name,rfb_date,canal_venta,cif,alta_month,alta_year,Family,Velocity,isNeba,Total
0,3-NOTRIAL,02i0N000006M6bnQAC,X Music4B,2019-02-12 00:00:01,dho,B85204915,2,2019,X Music,,0,1


In [388]:
alta_fam_ass = altas[['Total']].groupby(by=[altas.alta_year,altas.alta_month,altas.Family]).sum().reset_index()

In [389]:
alta_fam_ass.pivot_table('Total','Family',['alta_year','alta_month'], fill_value= 0, margins=True, aggfunc= sum)

alta_year    2018         2019                                               \
alta_month     10  11  12    1   2   3   4   5    6    7   8    9   10   11   
Family                                                                        
X Fiber         0   1   7    8   5   5   6   7   16   15   6   10    5    3   
X Music         3   2   3    4   7   2   0   6    0    0   0    5    0    0   
X Privacy       0   3   2    2   2  11  14   0   21    2   6    7   10    0   
X Protection    0   0   4    2   5  25  12   5   29   14  11   19   28   28   
X Security      0   0   0    2   1   0   0   2    2    8   1    4   25    6   
X UCom          0   5  16   19  23  12  17  17   46   72  14   56   66  100   
All             3  11  32   37  43  55  49  37  114  111  38  101  134  137   

alta_year         2020   All  
alta_month     12    1        
Family                        
X Fiber         6    5   105  
X Music         0    1    33  
X Privacy       0    2    82  
X Protection    4    0   186  
X Security     23    1    75  
X UCom        106   58   627  
All           139   67  1108

In [390]:
alta_fam_cif = functions.altas_cif(altas,'Family').pivot_table('cif','Family',['alta_year','alta_month'], fill_value= 0)
alta_fam_cif

alta_year    2018       2019                                         2020
alta_month     10 11 12   1  2  3  4  5   6   7   8   9   10  11  12   1 
Family                                                                   
X Fiber         0  1  6    7  5  3  6  5  11   7   3   8   1   1   0    2
X Music         3  2  3    2  4  1  0  6   0   0   0   3   0   0   0    1
X Privacy       0  1  1    1  1  4  7  0   6   1   2   3   3   0   0    1
X Protection    0  0  4    2  5  5  6  5  15  10  10  11  15  11   4    0
X Security      0  0  0    2  1  0  0  1   1   4   1   0   3   1   1    0
X UCom          0  1  5    3  8  5  5  7  11  19   7  17  24  27  20   20

In [391]:
alta_cif = functions.altas_cif(altas,'Total').T
alta_cif

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
alta_year,2018,2018,2018,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2020
alta_month,10,11,12,1,2,3,4,5,6,7,8,9,10,11,12,1
cif,3,5,18,13,20,14,18,21,36,36,21,40,44,39,25,23


In [25]:
# dic = altas['assetid'][(altas['alta_year']==2019) & (altas['alta_month']==12)]
# dic.to_csv('./comp_2019_12.csv',index=False)

# BAJAS

**MIRAR** El asset 02i0N00000I3wpuQAB no tienen el campo assehijo_mig informado

**FILTROS**

- Filtro 1: si migrado == 1 then segmento == 4-MIGRADO
- Filtro 2: si segmento == 4-MIGRADO rfb_date == rfb_migration

Tipos: 

1. Nivel asset - no hace falta tener en cuenta las altas
2. Nivel cliente/familia -  hace falta tener en cuenta las altas, una baja de un producto de un cliente se considerará cuando un cliente haya dado de baja todos los productos de una misma familua 
3. Nivel cliente - tendrá en cuenta como baja si un cliente ha dado de baja todos los productos de su cartera


¿Cómo trabajo para llegar al modelo altas - bajas?
- Tengo en cuenta sólo los assets del segmento 3-NOTRIAL
- Para aquéllos assets migrados se tendrá en cuenta la fecha rfb 'fabricada'

In [365]:
def rfb(row):
    return row['rfb_migration'] if row['migrado'] == '1' else row['rfb_date']

In [366]:
def baja__c(row):
    return 0 if row['deactivation_date'] is pd.NaT else -1

In [367]:
def bajas(cur):
    cur.execute('''
            select segmento, assetid, product_name, migrado,rfb_date, rfb_migration, deactivation_date, canal_venta, cif
            from cartera_xbo
            where process_date = (
            select max(process_date) from cartera_xbo)
            and segmento in ('3-NOTRIAL')
            and upper(product_name) not like '%CENTRALITA%'
            and rfb_date notnull
            ''')
    df = pd.DataFrame(cur.fetchall())
    col_names = []
    for e in range(len(cur.description)):
        col_names.append(cur.description[e][0])
    df.rename(columns = dict(zip(list(range(33)), col_names)), inplace = True)
    df['rfb__c'] = df.apply(lambda row: rfb(row), axis = 1)
    df.drop(columns=['rfb_date','rfb_migration'], axis=1, inplace=True)
    df['Family'] = df.product_name.apply(lambda x: functions.getFamilyFrom(x))
    df['Velocity'] = df.product_name.apply(lambda x: functions.getVelocityFrom(x))
    df['isNeba'] = df.product_name.apply(lambda x: functions.isNeba(x))
    df['Total'] = 1
    df['Bajas__c'] = df.apply(lambda row: baja__c(row), axis = 1)
    df['assets_act'] = df['Total'] + df['Bajas__c']
    return df

In [368]:
df = bajas(cur)

In [369]:
def deactivation(row):
    return pd.NaT if row['isActive'] >= 1 else row['deactivation_date']

In [370]:
def altas_bajas(df):
#     assets = df.Total.groupby(by=[df.cif,df.canal_venta]).sum().reset_index()
#     assets_act = df.assets_act.groupby(by=[df.cif,df.canal_venta]).sum().reset_index()
    altas = df.rfb__c.groupby(by=[df.cif,df.canal_venta]).min().reset_index()
    bajas = df.deactivation_date.groupby(by=[df.cif,df.canal_venta]).max().reset_index()
    final = pd.merge(pd.merge(pd.merge(assets_act,altas,on=['cif','canal_venta']),bajas,on=['cif','canal_venta']),assets,on=['cif','canal_venta'])
    final['isActive'] = final.assets_act.apply(lambda x: 1 if x>=1 else 0)
    final['deactivation_date'] = final.apply(lambda row: deactivation(row), axis = 1)
    final['assets_dea'] = final['Total'] - final['assets_act']
    final['isBaja'] = -(final['isActive']-1)
    final['Client'] = 1
    final.rename(columns={'Total':'assets_tot',
                      'rfb__c':'rfb',
                      }, inplace = True)
    return final

In [380]:
def altas_bajas_mes(altas_bajas_df):
    altas = altas_bajas_df[['Client']].groupby(by=[altas_bajas_df.rfb]).sum()
    bajas = altas_bajas_df[['isBaja']].groupby(by=[altas_bajas_df.deactivation_date]).sum()
    bajas.index.name = 'rfb'
    altas_bajas = altas.join(bajas, how='outer').fillna(0.0)
    mes = altas_bajas.groupby(by=[altas_bajas.index.year, altas_bajas.index.month]).sum()
    mes.index.names = ['year', 'month']
    mes.reset_index(inplace=True)
    return mes

In [381]:
def altas_bajas_mes_canal(altas_bajas_df):
    altas = altas_bajas_df[['Client']].groupby(by=[altas_bajas_df.rfb, altas_bajas_df.canal_venta]).sum()
    bajas = altas_bajas_df[['isBaja']].groupby(by=[altas_bajas_df.deactivation_date,altas_bajas_df.canal_venta]).sum()
    bajas.index.names = ['rfb','canal_venta']
    altas_bajas = altas.join(bajas, how='outer').fillna(0.0)
    mes_canal = altas_bajas.groupby(by=[altas_bajas.index.get_level_values('rfb').year, altas_bajas.index.get_level_values('rfb').month, altas_bajas.index.get_level_values('canal_venta')]).sum()
    mes_canal.index.names = ['year', 'month', 'canal']
    mes_canal.reset_index(inplace=True)
    return mes_canal

In [382]:
final = altas_bajas(df)

In [383]:
final.head(3)

,cif,canal_venta,assets_act,rfb,deactivation_date,assets_tot,isActive,assets_dea,isBaja,Client
0,03817373V,Venta Web,0,2019-11-20 00:00:01,2019-11-20 15:29:08,2,0,2,1,1
1,07477239P,dho,0,2018-10-25 09:38:28,2019-01-21 09:13:32,1,0,1,1,1
2,11058321J,dho,1,2019-01-23 00:00:00,NaT,1,1,0,0,1


In [386]:
altas_bajas_mes_df = altas_bajas_mes(final)
altas_bajas_mes_df

,year,month,Client,isBaja
0,2018,10,3.0,0.0
1,2018,11,5.0,0.0
2,2018,12,18.0,1.0
3,2019,1,13.0,3.0
4,2019,2,20.0,0.0
5,2019,3,14.0,2.0
6,2019,4,18.0,6.0
7,2019,5,21.0,8.0
8,2019,6,36.0,5.0
9,2019,7,37.0,4.0


In [385]:
altas_bajas_mes_canal_df = altas_bajas_mes_canal(final)
altas_bajas_mes_canal_df.head(3)

,year,month,canal,Client,isBaja
0,2018,10,Venta Web,1.0,0.0
1,2018,10,dho,2.0,0.0
2,2018,11,Venta Web,2.0,0.0


## TERMINAR CROSSTAB

In [315]:
h = pd.crosstab([mes_canal.year, mes_canal.month],mes_canal.canal,mes_canal.Client, aggfunc='sum').fillna(0).reset_index()

In [316]:
h[h['Venta Partner']>0]

canal,year,month,Venta Partner,Venta Web,dho,integrador
2,2018,12,1.0,6.0,8.0,3.0


In [219]:
# Tablas intermendias que contienen las altas y bajas por asset por mes.

a = df[['Total']].groupby(by=[df.rfb__c.dt.year,df.rfb__c.dt.month ]).sum()
b = df[['Total']].groupby(by=[df.deactivation_date.dt.year,df.deactivation_date.dt.month ]).sum()

# b = b.astype({'alta_month': 'int64', 'alta_year':'int64'})
# a = df[['Total']].groupby(by=[df.rfb__c.dt.year]).sum()
# b = df[['Total']].groupby(by=[df.deactivation_date.dt.year]).sum()

In [220]:
a.rename(columns = {'Total':'Altas'}, inplace=True)
b.rename(columns = {'Total':'Bajas'}, inplace=True)

In [221]:
# Cambio de tipo de datos de la tabla de bajas

b_i= b.index.get_level_values(0).astype('int64')
b_ii= b.index.get_level_values(1).astype('int64')

b.index = [b_i,b_ii] 

In [222]:
display(a.T)
display(b.T)

rfb__c 2018         2019                                                    \
rfb__c   10  11  12   1   2   3   4   5    6    7   8    9    10   11   12   
Altas     3  11  32   37  43  55  49  37  114  111  38  101  134  137  138   

rfb__c 2020  
rfb__c   1   
Altas    67

deactivation_date 2018 2019                                           2020
deactivation_date   12   1  2  3   4   5   6   7   8   9   10  11  12   1 
Bajas                1    6  2  9  11  12  14  18  24  27  25  20  20   21

In [223]:
b.index.levels[0].rename('year',inplace=True)
b.index.levels[1].rename('month',inplace=True)
a.index.levels[0].rename('year',inplace=True)
a.index.levels[1].rename('month',inplace=True)

In [224]:
a_b = a.join(b, how='outer').fillna(0)

In [225]:
a_b.Bajas = a_b.Bajas.astype('int64')

### TOTAL BAJAS ASSETS

In [226]:
a_b.T

year  2018         2019                                                   2020
month   10  11  12   1   2   3   4   5    6    7   8    9    10   11   12   1 
Altas    3  11  32   37  43  55  49  37  114  111  38  101  134  137  138   67
Bajas    0   0   1    6   2   9  11  12   14   18  24   27   25   20   20   21

### TOTAL BAJAS CLIENTES

In [315]:
df.head(2)

,segmento,assetid,product_name,migrado,deactivation_date,canal_venta,cif,rfb__c,Family,Velocity,isNeba,Total,Bajas__c
0,3-NOTRIAL,02i0N000006M6bnQAC,X Music4B,None,NaT,dho,B85204915,2019-02-12 00:00:01,X Music,,0,1,-1
1,3-NOTRIAL,02i0N000006M6boQAC,X Music4B,None,NaT,dho,B85204915,2019-02-12 00:00:01,X Music,,0,1,-1


In [235]:
a_2 = df[['Total']].groupby(by=[df.rfb__c.dt.year,df.rfb__c.dt.month,df.cif ]).sum()
b_2 = df[['Bajas__c']].groupby(by=[df.deactivation_date.dt.year,df.deactivation_date.dt.month,df.cif ]).sum()

In [237]:
# a_2.rename(columns = {'Total':'Altas'}, inplace=True)
# b_2.rename(columns = {'Total':'Bajas'}, inplace=True)

In [238]:
b_i_2= b_2.index.get_level_values(0).astype('int64')
b_ii_2= b_2.index.get_level_values(1).astype('int64')

b_2.index = [b_i_2,b_ii_2,b_2.index.get_level_values(2)] 

In [239]:
b_2.index.levels[0].rename('year',inplace=True)
b_2.index.levels[1].rename('month',inplace=True)
a_2.index.levels[0].rename('year',inplace=True)
a_2.index.levels[1].rename('month',inplace=True)

In [240]:
a_b_2 = a_2.join(b_2, how='outer').fillna(0)
a_b_2.Bajas__c = a_b_2.Bajas__c.astype('int64')

In [303]:
a_b_2[(a_b_2.index.get_level_values(2)=='21659638D')].cumsum(axis=0)

Total  Bajas__c
year month cif                       
2019 6     21659638D    1.0       0.0
     8     21659638D    2.0      -1.0
     12    21659638D    2.0      -2.0

In [302]:
a_b_2.cumsum(axis=0)

Total  Bajas__c
year month cif                        
2018 10    07477239P     1.0       0.0
           B86662517     2.0       0.0
           B86984325     3.0       0.0
     11    33571288M     4.0       0.0
           B28501229     9.0       0.0
...                      ...       ...
2020 1     B98944457  1094.0    -210.0
           F23527708  1099.0    -210.0
           G18559419  1104.0    -210.0
           G38796579  1106.0    -210.0
           G88558333  1107.0    -210.0

[515 rows x 2 columns]

In [242]:
df[df['cif']=='21659638D']

,segmento,assetid,product_name,migrado,deactivation_date,canal_venta,cif,rfb__c,Family,Velocity,isNeba,Total,Bajas__c
208,3-NOTRIAL,02i0N00000JbNf7QAF,X Protection,None,2019-08-21 10:57:26,dho,21659638D,2019-06-14 00:00:01,X Protection,,0,1,-1
662,3-NOTRIAL,02i0N00000NCGPyQAP,X Protection,None,2019-12-16 09:22:57,dho,21659638D,2019-08-21 00:00:01,X Protection,,0,1,-1


In [244]:
b_2[b_2.index.get_level_values(2)=='21659638D']

Bajas__c
year month cif                
2019 8     21659638D        -1
     12    21659638D        -1

In [248]:
b_2.rename(columns = {'Bajas__c':'Total'}, inplace=True)

In [249]:
a_b_concatenated = pd.concat([a_2, b_2], axis = 0, sort= False)

In [250]:
a_b_concatenated.head(2)

Total
year month cif             
2018 10    07477239P      1
           B86662517      1

In [251]:
a_b_concatenated[a_b_concatenated.index.get_level_values(2)=='21659638D']

Total
year month cif             
2019 6     21659638D      1
     8     21659638D      1
           21659638D     -1
     12    21659638D     -1

In [254]:
new = a_b_concatenated.reset_index()

In [267]:
new[new['cif']=='21659638D']

,year,month,cif,Total
130,2019,6,21659638D,1
220,2019,8,21659638D,1
498,2019,8,21659638D,-1
544,2019,12,21659638D,-1


In [355]:
new
new_2 = new[['Total']].groupby(by=[new.year,new.month,new.cif]).sum()

In [356]:
new_2

Total
year month cif             
2018 10    07477239P      1
           B86662517      1
           B86984325      1
     11    33571288M      1
           B28501229      5
...                     ...
2020 1     B98944457      1
           F23527708      5
           G18559419      5
           G38796579      2
           G88558333      1

[515 rows x 1 columns]

In [287]:
new_2[new_2.index.get_level_values(2)=='B73866469']

,,,Total
year,month,cif,
2020,1,B73866469,0


In [277]:
x = pd.crosstab(new.cif, [new.year, new.month], new.Total, aggfunc='sum')

In [297]:
prueba = x.fillna(0).cumsum(axis=1)

In [298]:
prueba

year      2018           2019                                               \
month       10   11   12   1    2    3    4    5    6    7    8    9    10   
cif                                                                          
03817373V  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
07477239P  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11058321J  0.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
12371541W  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
16601899Q  0.0  0.0  0.0  0.0  0.0  0.0  0.0  5.0  5.0  3.0  3.0  3.0  3.0   
...        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
P3602100D  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  5.0  5.0  5.0   
R2800701A  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  4.0  4.0   
R2800902E  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  2.0   
R3300154F  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
W4002909B  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  2.0  2.0  2.0   

year                2020  
month       11   12   1   
cif                       
03817373V  0.0  0.0  0.0  
07477239P  0.0  0.0  0.0  
11058321J  1.0  1.0  1.0  
12371541W  0.0  0.0  0.0  
16601899Q  3.0  1.0  1.0  
...        ...  ...  ...  
P3602100D  5.0  5.0  5.0  
R2800701A  4.0  4.0  4.0  
R2800902E  2.0  2.0  2.0  
R3300154F  1.0  0.0  0.0  
W4002909B  2.0  2.0  2.0  

[375 rows x 16 columns]

In [296]:
# prueba.to_csv('./prueba.csv')

In [305]:
df.dtypes

segmento                     object
assetid                      object
product_name                 object
migrado                      object
deactivation_date    datetime64[ns]
canal_venta                  object
cif                          object
rfb__c               datetime64[ns]
Family                       object
Velocity                     object
isNeba                        int64
Total                         int64
Bajas__c                      int64
dtype: object

In [316]:
d = df.drop(columns=['assetid', 'product_name', 'migrado', 'canal_venta', 'Velocity', 'isNeba','segmento','rfb__c','Total'])

In [317]:
e = df.drop(columns=['assetid', 'product_name', 'migrado', 'canal_venta', 'Velocity', 'isNeba','segmento','deactivation_date','Bajas__c']) 

In [321]:
e.rename(columns = {'rfb__c':'date'}, inplace=True)

In [323]:
d.rename(columns = {'Bajas__c':'Total','deactivation_date':'date'}, inplace=True)

In [328]:
d.dropna(inplace = True)

In [332]:
f = pd.concat([e,d], ignore_index=True, sort=True)

In [347]:
f['month'] = f.date.apply(lambda x: x.month)

In [370]:
g = f[['Total']].groupby(by=[f.cif,f.date.dt.year,f.month]).sum().reset_index()
# f.groupby(f.cif).sum()
g.head(3)

,cif,date,month,Total
0,03817373V,2019,11,0
1,07477239P,2018,10,1
2,07477239P,2019,1,-1


In [389]:
x = pd.crosstab(g.cif, [g.date, g.month], g.Total, aggfunc='sum')

In [385]:
x.reset_index(inplace=True)

In [421]:
x.fillna('', inplace=True)

In [422]:
x.head()

date      2018       2019                                  2020
month       10 11 12   1  2  3  4  5  6  7  8  9  10 11 12   1 
cif                                                            
03817373V                                             0        
07477239P    1         -1                                      
11058321J               1                                      
12371541W          1   -1                                      
16601899Q                           5    -2             -2

In [423]:
x['total'] = x.sum(axis=1)

In [425]:
x.dtypes

date   month
2018   10        object
       11        object
       12        object
2019   1         object
       2         object
       3         object
       4         object
       5         object
       6         object
       7         object
       8         object
       9         object
       10        object
       11        object
       12        object
2020   1         object
total           float64
dtype: object

In [401]:
x.loc['16601899Q']

date  month
2018  10       NaN
      11       NaN
      12       NaN
2019  1        NaN
      2        NaN
      3        NaN
      4        NaN
      5        5.0
      6        NaN
      7       -2.0
      8        NaN
      9        NaN
      10       NaN
      11       NaN
      12      -2.0
2020  1        NaN
Name: 16601899Q, dtype: float64

In [410]:
y =x.cumsum(axis=1)

In [415]:
y.cum

date      2018          2019                                                \
month       10  11   12   1    2   3   4    5   6    7    8    9   10   11   
cif                                                                          
03817373V  NaN NaN  NaN  NaN  NaN NaN NaN  NaN NaN  NaN  NaN  NaN NaN  0.0   
07477239P  1.0 NaN  NaN  0.0  NaN NaN NaN  NaN NaN  NaN  NaN  NaN NaN  NaN   
11058321J  NaN NaN  NaN  1.0  NaN NaN NaN  NaN NaN  NaN  NaN  NaN NaN  NaN   
12371541W  NaN NaN  1.0  0.0  NaN NaN NaN  NaN NaN  NaN  NaN  NaN NaN  NaN   
16601899Q  NaN NaN  NaN  NaN  NaN NaN NaN  5.0 NaN  3.0  NaN  NaN NaN  NaN   
...        ...  ..  ...  ...  ...  ..  ..  ...  ..  ...  ...  ...  ..  ...   
P3602100D  NaN NaN  NaN  NaN  NaN NaN NaN  NaN NaN  NaN  5.0  NaN NaN  NaN   
R2800701A  NaN NaN  NaN  NaN  NaN NaN NaN  NaN NaN  NaN  NaN  4.0 NaN  4.0   
R2800902E  NaN NaN  NaN  NaN  NaN NaN NaN  NaN NaN  NaN  NaN  2.0 NaN  NaN   
R3300154F  NaN NaN  NaN  NaN  1.0 NaN NaN  NaN NaN  NaN  NaN  NaN NaN  NaN   
W4002909B  NaN NaN  NaN  NaN  NaN NaN NaN  NaN NaN  2.0  NaN  NaN NaN  NaN   

date           2020  
month       12   1   
cif                  
03817373V  NaN  NaN  
07477239P  NaN  NaN  
11058321J  NaN  NaN  
12371541W  NaN  NaN  
16601899Q  1.0  NaN  
...        ...  ...  
P3602100D  NaN  NaN  
R2800701A  NaN  NaN  
R2800902E  NaN  NaN  
R3300154F  0.0  NaN  
W4002909B  NaN  NaN  

[375 rows x 16 columns]

# TOTALES

In [494]:
def rfb(row):
    return row['rfb_migration'] if row['migrado'] == '1' else row['rfb_date']
def purchase(row):
    return row['purchase_migration'] if row['migrado'] == '1' else row['purchase_date']
def purchaseToAsset(row):
    return 0 if row['rfb__c'] is pd.NaT else 1
def baja__c(row):
    return 0 if row['deactivation_date'] is not pd.NaT else -1
def cancelledSales(row):
    return 1 if row['rfb__c'] is pd.NaT else 0
def ventas(cur):
    cur.execute('''
            select segmento, assetid, cif, canal_venta, product_name, 
                    migrado,rfb_date, rfb_migration, deactivation_date, cancellation_date,
                    purchase_date, purchase_migration, deactivation_request_date
            from cartera_xbo
            where process_date = (
            select max(process_date) from cartera_xbo)
            and segmento in ('3-NOTRIAL')
            and upper(product_name) not like '%CENTRALITA%'
            ''')
    df = pd.DataFrame(cur.fetchall())
    col_names = []
    for e in range(len(cur.description)):
        col_names.append(cur.description[e][0])
    df.rename(columns = dict(zip(list(range(33)), col_names)), inplace = True)
    df['rfb__c'] = df.apply(lambda row: rfb(row), axis = 1)
    df.drop(columns=['rfb_date','rfb_migration'], axis=1, inplace=True)
    df['purchase_date__c'] = df.apply(lambda row: purchase(row), axis = 1)
    df.drop(columns=['purchase_migration','purchase_date'], axis=1, inplace=True)
    df['Family'] = df.product_name.apply(lambda x: functions.getFamilyFrom(x))
    df['Velocity'] = df.product_name.apply(lambda x: functions.getVelocityFrom(x))
    df['isNeba'] = df.product_name.apply(lambda x: functions.isNeba(x))
    df['isSold'] = 1
    df['isAsset'] =  df.apply(lambda row: purchaseToAsset(row), axis = 1)
    df['isBaja'] = df.apply(lambda row: baja__c(row), axis = 1)
    df['isActive'] = df['isAsset'] + df['isBaja']
    df['isCancelled'] = df.apply(lambda row: cancelledSales(row), axis = 1)
    to_drop = df[(df['rfb__c'].isnull())&((df['deactivation_date'].notnull())|(df['deactivation_request_date'].notnull()))].index
    df.drop(to_drop , inplace=True)
    df.drop(columns=['deactivation_request_date'], axis=1, inplace=True)
    return df

In [495]:
df = ventas(cur)

In [496]:
df.head(1)

,segmento,assetid,cif,canal_venta,product_name,migrado,deactivation_date,cancellation_date,rfb__c,purchase_date__c,Family,Velocity,isNeba,isSold,isAsset,isBaja,isActive,isCancelled
0,3-NOTRIAL,02i0N000006M6bnQAC,B85204915,dho,X Music4B,None,NaT,NaT,2019-02-12 00:00:01,2019-02-12 00:00:01,X Music,,0,1,1,-1,0,0


In [462]:
 df[(df['rfb__c'].isnull())&(df['cancellation_date'].notnull())]

,segmento,assetid,cif,canal_venta,product_name,migrado,deactivation_date,cancellation_date,rfb__c,purchase_date__c,Family,Velocity,isNeba,isSold,isAsset,isBaja,isActive
68,3-NOTRIAL,02i0N00000I2rSoQAJ,B87414470,dho,X Fiber 50 Mbps,None,NaT,2018-12-19 17:24:38,NaT,2018-12-14 00:00:01,X Fiber,50 Mbps,0,1,0,0,0
94,3-NOTRIAL,02i0N00000I3NZNQA3,B88210828,integrador,X Fiber 100 Mbps,None,NaT,2019-01-11 10:36:41,NaT,2019-01-09 00:00:01,X Fiber,100 Mbps,0,1,0,0,0
95,3-NOTRIAL,02i0N00000I3NZhQAN,B88210828,integrador,X Security 100,None,NaT,2019-01-17 10:38:42,NaT,2019-01-09 00:00:01,X Security,100 Mbps,0,1,0,0,0
138,3-NOTRIAL,02i0N00000JalwOQAR,P2808500I,dho,X Fiber 200 Mbps,None,NaT,2019-06-24 13:49:19,NaT,2019-05-30 00:00:01,X Fiber,200 Mbps,0,1,0,0,0
256,3-NOTRIAL,02i0N00000KmANgQAN,B84562305,dho,X Fiber 100 Mbps,None,NaT,2019-05-21 16:56:40,NaT,2019-05-16 00:00:01,X Fiber,100 Mbps,0,1,0,0,0
263,3-NOTRIAL,02i0N00000KmHcTQAV,B78909611,dho,X UCom Licencia de Colaboración,None,NaT,2019-05-24 16:25:30,NaT,2019-05-21 00:00:01,X UCom,,0,1,0,0,0
264,3-NOTRIAL,02i0N00000KmHcUQAV,B78909611,dho,X UCom Licencia de Colaboración,None,NaT,2019-05-24 16:26:35,NaT,2019-05-21 00:00:01,X UCom,,0,1,0,0,0
267,3-NOTRIAL,02i0N00000KmHebQAF,B82711284,dho,X Privacy 200,None,NaT,2019-05-21 11:12:26,NaT,2019-05-21 00:00:01,X Privacy,200 Mbps,0,1,0,0,0
268,3-NOTRIAL,02i0N00000KmHecQAF,B82711284,dho,X Privacy 200,None,NaT,2019-05-21 11:12:47,NaT,2019-05-21 00:00:01,X Privacy,200 Mbps,0,1,0,0,0
285,3-NOTRIAL,02i0N00000KmOAXQA3,B71264451,dho,X UCom Licencia de Colaboración,None,NaT,2019-06-07 07:19:50,NaT,2019-05-23 00:00:01,X UCom,,0,1,0,0,0


In [450]:
ventas = df.purchase_date__c.groupby(by=[df.cif,df.canal_venta]).min().reset_index()

In [451]:
ventas.head()

,cif,canal_venta,isSold
0,,Venta Web,1
1,03817373V,Venta Web,2
2,07477239P,dho,1
3,11058321J,dho,1
4,12371541W,Venta Web,1
